In [1]:
include("optimization_library.jl");

# Exercise 8.1: Optimal Path for Santa

<img src="figures/santa-claus.png" width="300">

In [2]:
# ================================================================================================
# vector c, matrix A and vector b

c = [4857; 3281; 7891; 7891; 7312; 7312; 9892; 9892]

A = [1 1 0 0 0 0 0 0;
    0 1 1 -1 0 0 1 -1;
    1 0 -1 1 1 -1 0 0;
    0 0 0 0 -1 1 -1 1]

d_eu = 78
d_usa = 60
d_bz = 40
supply = d_eu + d_usa + d_bz

b = [supply; d_eu; d_usa; d_bz]
# ================================================================================================

4-element Vector{Int64}:
 178
  78
  60
  40

In [3]:
# ================================================================================================
# 1. Implement the objective function, its gradient and Hessian.
# 2. Implement the functions g(x) for the inequality constraints, their gradients and Hessians.
nx = size(A,2)

f(x) = c' * x
df(x) = c
Hf(x) = zeros(nx,nx)

g = Vector(undef, nx)
dg = Vector(undef, nx)
Hg = Vector(undef, nx)
    
for i in 1:nx # allow only positive quantities
    g[i] = x -> -x[i]
    dg[i] = x -> begin d = zeros(nx); d[i] = -1; return d end
    Hg[i] = x -> zeros(nx, nx)
end

In [4]:
# ================================================================================================
# Apply the function ConstraintElimination_ipm() from the optimization library
x_best, trace = ConstraintElimination_ipm(f,df,Hf,A,b,g=g,dg = dg, Hg = Hg, eps = 0.00001, barrier_increase = 2.0)
# ================================================================================================

10.856831315937054


([99.99996889093254, 78.00003110906742, 3.300940075945391e-6, 4.948535301974033e-6, 2.1368983649239226e-6, 39.99997267542617, 1.6640043210003341e-6, 3.1125476521509654e-5], [-5.783196721259053 -20.320764434307378 … -20.321946072253887 -20.32198546112234; 21.078327588890346 12.988572929023766 … 12.988684104674126 12.988687810690806; … ; 1.4198339628906264e6 1.0341345982253124e6 … 1.0341011538219411e6 1.0340996852151988e6; 0.0 101.0 … 101.0 101.0])

In [5]:
x_best

8-element Vector{Float64}:
 99.99996889093254
 78.00003110906742
  3.300940075945391e-6
  4.948535301974033e-6
  2.1368983649239226e-6
 39.99997267542617
  1.6640043210003341e-6
  3.1125476521509654e-5

In [6]:
println("average distance per package: ",round((c'*x_best)/supply,digits=2)," km")

average distance per package: 5809.54 km


## Howe to use the SCS solver

We will now learn how to use the SCS solver using the JuMP package. The JuMP package provides macros that allow to access some functions of the solver in a very convenient way.

In [7]:
using JuMP
using SCS

In [8]:
# ================================================================================================
# Uncomment the following lines to solve the problem with the SCS solver.
model = Model(SCS.Optimizer)
@JuMP.variable(model, x[1:nx])
@JuMP.constraint(model, A * x .== b)
@JuMP.constraint(model,  -x .<= zeros(nx))
@JuMP.objective(model, Min, c'*x)
JuMP.optimize!(model)
# ================================================================================================

----------------------------------------------------------------------------
	SCS v2.1.4 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 24, CG tol ~ 1/iter^(2.00)
eps = 1.00e-005, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-003
Variables n = 8, constraints m = 12
Cones:	primal zero / dual free vars: 4
	linear vars: 8
Setup time: 6.71e-005s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|1.85e+019 3.45e+019 1.00e+000 -5.04e+025 5.89e+025 4.00e+025 2.84e-005 
    40|9.75e-007 3.21e-006 1.15e-006 1.03e+006 1.03e+006 3.29e-010 2.43e-004 
---------------------------------------------------------------------

In [9]:
# ================================================================================================
# Uncomment the following line to retrieve the minimizer. Compare the solution to the result from the 
# ConstraintElimination_ipm() function.
sol = JuMP.value.(x)
# ================================================================================================

8-element Vector{Float64}:
 99.9999833732216
 78.00003936869459
 -2.3186648746859928e-5
  1.5582576144783267e-5
 -5.2352190441403705e-5
 40.00007195634038
 -7.297989673031612e-5
 -9.083558501392666e-5

In [10]:
abs.(sol-x_best) .< 0.01

8-element BitVector:
 1
 1
 1
 1
 1
 1
 1
 1

In [11]:
@assert sum(abs.(sol-x_best) .< 0.01) == length(x_best)

# Exercise 8.2: Truss Structure

In [12]:
using Luxor

# Function for drawing the truss structure
# J: Vector of joint coordinates
# B: Vector of beams, i.e. vector of tuples with joint indices that the beams connect
# Jc: Vector of cantilever joints
# Jl: Vector with load on each joint
# mb: Vector with beam masses
function drawTruss(J, B, Jc, Jl, mb)
    scale = 40
    function pos(j)
        return Point(J[j][1]*scale - 400, -J[j][2]*scale)
    end
    @pdf begin
        background("white")
        
        # Draw beams
        setcolor("black")
        for b=1:nB
            if mb[b] <= 1
                continue
            end
            setline(sqrt(mb[b]))
            line(pos(B[b][1]), pos(B[b][2]), :stroke)
        end
        
        # Draw road
        setcolor("blue")
        setline(5)
        line(Point(-400,0), Point(-400 + max_x*scale,0), :stroke)
        
        # Draw joints
        setcolor("red")
        for j = 1:length(J)
            if j in Jc
                circle(pos(j), 8, :fill)
            end
        end
        
        # Draw load arrows
        setcolor("green")
        for j = 1:length(J)
            if LinearAlgebra.norm(Jl[j]) <= 0.1
                continue
            end
            Luxor.arrow(pos(j), pos(j) - (Point(Jl[j][1], Jl[j][2])/5000), linewidth=5)
        end
    end 1000 400
end



drawTruss (generic function with 1 method)

In [13]:
# Vector with indices of joints. Each entry is a two-vector with the x-y coordinates
J = [[x,y] for x=0:1:17, y=-2:1:4]

# number of joints
nJ = length(J) 

# Constructing vector of beams of lenght 1 m. Each beam corresponds to a tuple with the indeces of the joints the beam is connecting
# Connect all joints that are in a radius < 2m.
rad = 2.0
B = [(i,j) for i=1:nJ, j = 1:nJ if j < i && LinearAlgebra.norm(J[i]-J[j]) < rad]

# number of beams
nB = length(B)

# Vector storing indices of beams connected to each joint
Bj = [[b for b=1:nB if B[b][1] == j || B[b][2] == j] for j=1:nJ];

# number of joints along x-axis
max_x = maximum([J[i][1] for i=1:nJ])

# Defining cantilever joints as the joints that are on both sides of the bridge and below or equal to the street level
Jc = [i for i=1:nJ if ((J[i][1] == 0 || J[i][1] == max_x) && J[i][2] <= 0)];
nC = length(Jc) # number of cantilever points

# external load excerted on each joint at street level
load = 20000
# External loads excerted on the joints at street level (y-coordinate equals zero)
Jl = [J[j][2] == 0 ? [0.0, -load*9.81] : [0.0,0.0] for j=1:nJ];

In [14]:
Jc

6-element Vector{Int64}:
  1
 18
 19
 36
 37
 54

In [15]:
# Initialize the masses of beams for drawing of initial structure
mb_init = ones(nB)*10;

# Draw initial truss structure
drawTruss(J,B,Jc,Jl,mb_init)

## The Optimization Problem

In lecture 5, slide 15, we learned that the weight of the truss structure defined above can be minimized by solving the problem under the constraint of a given load of 2000 kg distributed evenly on the joints on the street level: 

  \begin{equation*}
    \begin{aligned}
    \min_{\substack{f_1, \dots, f_n\\ \tilde f_1,\dots, \tilde f_n}} \quad & \sum_{b=1}^n \Big[ \delta \cdot  \|\vec d_b\| \cdot \tilde f_b  \Big]\\
    \text{subject to} \quad & 
    \vec l_j + \sum_{b \in B_j} \Big[ \frac{\vec d_b}{\|\vec d_b\|} f_b \Big] = \vec 0, & j = 1,\dots,m\\
    & \tilde f_b \geq f_b,\quad \tilde f_b \geq -f_b, & b = 1,\dots,n
    \end{aligned}
  \end{equation*}
  
This problem neglects the weight of the beams. Modify the problem to include the weight of the beams into the equality constraints. Use $\delta = 5\cdot 10^{-4}\,$kg/(Nm).
**Hint:** The gravitational force $f$ in the unit Newton depends on the mass $m$ by $f = 9.81 m\,$ and acts only into the negative y-direction.
Implement the problem as a linear program of the form:
\begin{align}
 \min_{\vec{x}} \quad & \vec{c}^\top \vec{x}\\
 \text{subject to} \quad & A\vec{x} = \vec{b}\\
 \quad & C\vec{x} \leq \vec{0},
\end{align}

with the target variable $x = [f_1, f_2, \dots, \tilde f_1, \tilde f_2, \dots]$. 

There are 2 * nB inequality constraints and  2 * (nJ - nC) equality constraints.

In [16]:
delta = 5e-4

# ================================================================================================
# Construct the vector c.
c = vcat(zeros(nB), delta*[LinearAlgebra.norm(J[b[1]]-J[b[2]]) for b in B]);
# ================================================================================================

In [17]:
println("Dimensions of the problem: ", length(c))
println("Inequalities: ", 2*nB)
println("Equality constraints: ", 2*(nJ-nC))

Dimensions of the problem: 862
Inequalities: 862
Equality constraints: 240


In [18]:
# Construct matrix A and vector b for the equality constraints.
A = zeros(2 * (nJ-nC), length(c));
b = zeros(2 * (nJ-nC))

idx = 0
# iterate over all joints
for j=1:nJ
    
    # ignore cantilever joints
    if j in Jc
        continue
    end
    
# ================================================================================================
    # Construct the vector b.
    # External load applied at the joint
    b[2*idx+1] = -Jl[j][1] # joint load in x-direction
    b[2*idx+2] = -Jl[j][2] # joint load in y-direction
# ================================================================================================
    
    # Iterate over all beams connected to joint j
    for b in Bj[j]
        db = J[B[b][1]] - J[B[b][2]] # beam direction vector
        
# ================================================================================================
        # Construct matrix A
        db_dir = db / LinearAlgebra.norm(db)
        A[2*idx+1,b] = db_dir[1] # force across the beam in x-direction --> odd rows of A
        A[2*idx+2,b] = db_dir[2] # force across the beam in y-direction --> even rows of A
        
        A[2*idx+2,nB + b] = -0.5*delta * 9.81*LinearAlgebra.norm(db) # gravitational force of each beam into the negative 
                                                                     # y-direction (depends on ftilde) 
# ================================================================================================
    end
    idx = idx + 1
end

In [19]:
# Inequality constraints Cx <= d
C = zeros(2 * nB, length(c))
d = zeros(2 * nB);

# ================================================================================================
# Fill the matrix C
for b = 1:nB
    # fb - tildefb <= 0 --> odd rows of C
    C[2*b-1,b] = 1.0
    C[2*b-1,b+nB] = -1.0
    
    # -fb - tildefb <= 0 --> even rows of C
    C[2*b,b] = -1.0
    C[2*b,b+nB] = -1.0
end
# ================================================================================================

## Use the solver to find the solution

In [20]:
model = JuMP.Model(SCS.Optimizer)
@variable(model, x[1:2*nB])
@constraint(model, A * x .== b)
@constraint(model, C * x .<= d)
@objective(model, Min, c'*x)
optimize!(model)

----------------------------------------------------------------------------
	SCS v2.1.4 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 3794, CG tol ~ 1/iter^(2.00)
eps = 1.00e-005, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 10, rho_x = 1.00e-003
Variables n = 862, constraints m = 1102
Cones:	primal zero / dual free vars: 240
	linear vars: 862
Setup time: 3.70e-003s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|1.78e+020 3.55e+018 1.00e+000 -8.06e+023 1.36e+024 7.58e+023 1.41e-003 
   100|3.36e-002 6.34e-004 2.68e-003 1.59e+004 1.60e+004 1.30e-013 1.08e-001 
   200|2.14e-002 3.67e-004 6.90e-004 1.62e+004 1.62e+004 1.

In [21]:
sol = value.(x)
masses = c.*sol
masses = masses[nB+1:2*nB]

drawTruss(J,B,Jc,Jl, masses)

In [22]:
sol

862-element Vector{Float64}:
       0.20133928060385975
       0.0
 -967804.964944478
  198612.24815880874
  141472.96706545405
       0.34664382366339164
 -139407.97896141387
       0.1388515114553606
  141474.1709769432
      -0.33754649901965794
 -139406.86537601112
     965.5820675679556
     687.7496611089657
       ⋮
      -0.008420770427207673
      -0.008114606865752662
      -0.0031714497788145474
      -0.008561031694966404
      -0.0031715749294733667
      -0.00811460675123798
      -0.00842077039662575
      -0.008731120349776576
      -0.008767835750387334
      -0.008667803152464267
      -0.008686598829361927
      -0.008688481765179838

In [23]:
masses

431-element Vector{Float64}:
  -4.872080217613236e-6
  -4.3434129428783905e-6
 684.3413923907893
  99.30590586790828
 100.0363831387471
  -4.815753512417478e-6
  98.57644260450765
  -4.123646336729575e-6
 100.03749700883358
  -3.944295407849726e-6
  98.57563602842656
   0.482845383437206
   0.4863365754577162
   ⋮
  -4.210385213603836e-6
  -4.057303432876331e-6
  -1.5857248894072737e-6
  -4.280515847483202e-6
  -1.5857874647366834e-6
  -4.05730337561899e-6
  -4.210385198312876e-6
  -4.3655601748882885e-6
  -4.383917875193667e-6
  -4.3339015762321335e-6
  -4.343299414680963e-6
  -4.344240882589919e-6